### Autoencoder

: 역전파를 이용해 입력 변수와 목적 변수를 동일한 값으로 넣고 다시 복원하려는 신경망 종류
- 오토인코더의 은닉 유닛들의 개수는 일반적으로 입력 변수보다 적으며, 이 특징은 인코더에게 디코더가 원복하려고 하는 입력 변수의 간결한 표현을 학습하게 강요한다. 입력 변수들 간의 상관관계가 있다면 결국 주성분 분석(PCA)를 사용해 학습된 것과 비슷하게 저차원의 테이터 표현이 된다.
- 인코더 구성 요소를 사용해 입력을 압축해 표현한다.
- Stacked autoencoder는 풍부한 표현력을 가지며 연속적인 표현 계층들은 합성곱 신경망에서의 합성곱과 풀링 연산 비슷하게 입력의 계층적인 그룹을 잡아낸다.
- denosing autoencoder
- variational autoencoder : http://alexadam.ca/ml/2017/05/05/keras-vae.html

In [ ]:
from keras.layers.core import Dense, Dropout, SpatialDropout1D
from keras.layers.convolutional import Conv1D
from keras.layers.embeddings import Embedding
from keras.layers.pooling import GlobalMaxPooling1D
from keras.models import Sequential
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import collections

In [ ]:
np.random.seed(2019)

In [ ]:
# from keras.datasets import reuters

from nltk.corpus import reuters
import numpy as np

In [ ]:
# words with frequency rank of 1-3000
(X_train, Y_train), (X_test, Y_test) = reuters.load_data(num_words=5000, test_split=0.2)

In [ ]:
X_train.shape

In [ ]:
category = np.max(Y_train)+1
category

In [ ]:
MAX_LEN = 500
vocab_size = 5000
embed_size = MAX_LEN
window_size = 1
BATCH_SIZE = 100
NUM_EPOCHS= 20

LATENT_SIZE = 10

In [ ]:
from nltk.corpus import stopwords

def trim(word):
    word = word.lower().strip()
    if word not in stopwords:
        return word
    else :
        return ''
    
def build_vocab(vocab_size):
    words = reuters.words()
    words = list(filter(lambda x: x.strip(), map(lambda x: trim(x), words)))
    
build_vocab(vocab_size)

In [ ]:
from keras.preprocessing import sequence

x_train = sequence.pad_sequences(X_train, maxlen=MAX_LEN)
x_test = sequence.pad_sequences(X_test, maxlen=MAX_LEN)

In [ ]:
from keras.utils import np_utils

y_train = np_utils.to_categorical(Y_train)
y_test = np_utils.to_categorical(Y_test)

### Autoencoder

In [ ]:
from keras.layers import Input
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import Bidirectional
from keras.layers.core import RepeatVector
from keras.models import Model

inputs = Input(shape=(MAX_LEN, embed_size), name='input')
encoded = Bidirectional(LSTM(LATENT_SIZE), merge_mode='sum', name='encoder_lstm')(inputs)
decoded = RepeatVector(MAX_LEN, name='repeater')(encoded)
decoded = Bidirectional(LSTM(embed_size, return_sequences=True), merge_mode='sum', name='dncoder_lstm')(decoded)

autoencoder = Model(inputs, decoded)
autoencoder.compile(optimizer='sgd', loss='mse')

In [ ]:
hiotory = autoencoder.fit(x_train,\
#                                    steps_per_epoch=BATCH_SIZE,\
                                   epochs=NUM_EPOCHS,\
                                   validation_data=x_test)
#                                    validation_steps=BATCH_SIZE)

오코인코더를 실행해 예측한 임베딩을 구한다. 코사인유사도를 사용해 두 벡터를 비교한다.

In [ ]:
encoder = Model(autoencoder.input, autoencoder.get_layer('encoder_lstm').output)

In [ ]:
def compute_cosine_similarity(x, y):
    return np.dot(x, y) / (np.linalg.norm(x, 2) * np.linalg.norm(y, 2))

In [ ]:
k = 500
cosims = np.zeros((k))
i = 0

for i in range(len(x_test)):
    x= x_test[i]
    y = Y_test[i]
    
    _y = autoencoder.predict(x)
    x_vec = encoder.predict(x)
    y_vec = encoder.predict(_y)
    
    for rid in range(x_vec.shape[0]):
        if i>=k:
            break
        cosims[i] = compute_cosine_similarity(x_vec[rid], y_vec[rid])
        
        if i <= 10:
            print(cosims[i])
            i +=1
        if i >= k:
            break